In [ ]:
import os
import sys
import copy
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
sys.path.append(os.path.join(os.path.pardir, 'gotmtool'))
sys.path.append(os.path.join(os.path.pardir, 'lesview'))
from gotmtool import *
from lesview import *

In [ ]:
# casename = 'R7_B 05WV00_ST00_ens01'
# ncarles_file = 'his.mp.vis.000001.028801.nc'
casename = 'R7_BF05WD10WV00_ST00_ens01'
ncarles_file = 'his.mp.vis.000001.036001.nc'
turbmethod = 'KPPLT-LF17'
# turbmethod = 'KPP-CVMix'
# turbmethod = 'GLS-C01A'

shortname = casename[3:15]
print(shortname)
ncarles_dir  = os.path.join(os.path.pardir, 'data', 'ncarles', 'LF17_Profile_data', '{:s}'.format(casename))
ncarles_pfl  = NCARLESDataProfile(filepath=os.path.join(ncarles_dir, ncarles_file))
ncarles_ds   = ncarles_pfl.dataset

gotm_dir   = os.path.join(os.path.pardir, 'gotm', 'run', 'constant_forcing')
gotm_sim1  = Simulation(path=os.path.join(gotm_dir, shortname, turbmethod))
gotm_sim2  = Simulation(path=os.path.join(gotm_dir, shortname, '{:s}_Relax'.format(turbmethod)))
gotm_ds1   = gotm_sim1.load_data()
gotm_ds2   = gotm_sim2.load_data()

In [ ]:
ncarles_ds

In [ ]:
gotm_ds1

In [ ]:
gotm_ds2

In [ ]:
ustar = 0.012 # m/s
Q0 = 5/1000/4200
alpha = 2e-4
g = 9.81
B0 = Q0*g*alpha
Tstar = Q0/ustar
NNT0 = 0.01

In [ ]:
# one inertial period (17 hours)
startdate = '2000-01-01T10:00:00'
enddate = '2000-01-02T03:00:00'
tslice = slice(startdate, enddate)

In [ ]:
def cmp_da(da0, da1, da2, var, units, levels, cmap):
    fig1, axarr = plt.subplots(3, 1, sharex='col')
    fig1.set_size_inches([9, 9])
    
    da0.plot(ax=axarr[0], levels=levels, cmap=cmap)
    da1.plot(ax=axarr[1], levels=levels, cmap=cmap)
    da2.plot(ax=axarr[2], levels=levels, cmap=cmap)
    
    depth = -100
    for j in np.arange(3):
        axarr[j].set_ylim([depth, 0])
        axarr[j].set_xlabel('')
        axarr[j].set_title('')
        axarr[j].set_ylabel('depth (m)')

    fig2 = plt.figure(figsize=[6,4])
    da0.sel(time=tslice).mean(dim='time').plot(y=da0.dims[0], color='k', linestyle='-', label='LES')
    da1.sel(time=tslice).mean(dim='time').plot(y=da1.dims[0], color='k', linestyle='--', label='GOTM')
    da2.sel(time=tslice).mean(dim='time').plot(y=da2.dims[0], color='k', linestyle=':', label='GOTM Relax')
    vmin = np.min(levels)
    vmax = np.max(levels)
    plt.ylim([depth, 0])
    plt.xlim([vmin, vmax])
    plt.xlabel(var+' ('+units+')')
    plt.ylabel('Depth (m)')
    plt.title('Time averaged')
    plt.legend()
    
    
def plot_mean_fields(var, units, levels, cmap, lesvar, gotmvar, lesshift=0, scale=1):
    da0 = (ncarles_ds.data_vars[lesvar]+lesshift)*scale
    da1 = gotm_ds1.data_vars[gotmvar].squeeze()*scale
    da2 = gotm_ds2.data_vars[gotmvar].squeeze()*scale
    for da in [da0, da1, da2]:
        da.attrs['long_name'] = var
        da.attrs['units'] = units
    cmp_da(da0, da1, da2, var, units, levels, cmap)


def get_flux(lam, num, gam):
    tmp = xr.zeros_like(num)
    nt = lam.shape[1]
    for i in np.arange(nt):
        tmp.data[1:-1,i] = (lam.data[:-1,i]-lam.data[1:,i])/(lam.z.data[:-1]-lam.z.data[1:])
    flux = - num * tmp + gam
    return flux
    
    
def plot_turbulent_flux(var, units, levels, cmap, lesvar1, lesvar2, gotmvar1, gotmvar2, gotmvar3, scale=1):
    da0 = (ncarles_ds.data_vars[lesvar1]+ncarles_ds.data_vars[lesvar2])*scale
    da1 = get_flux(gotm_ds1.data_vars[gotmvar1].squeeze(),
                   gotm_ds1.data_vars[gotmvar2].squeeze(),
                   gotm_ds1.data_vars[gotmvar3].squeeze())*scale
    da2 = get_flux(gotm_ds2.data_vars[gotmvar1].squeeze(),
                   gotm_ds2.data_vars[gotmvar2].squeeze(),
                   gotm_ds2.data_vars[gotmvar3].squeeze())*scale
    for da in [da0, da1, da2]:
        da.attrs['long_name'] = var
        da.attrs['units'] = units
    cmp_da(da0, da1, da2, var, units, levels, cmap)
    
# def plot_turbulent_flux(var, units, levels, cmap, lesvar1, lesvar2, gotmvar1, gotmvar2, gotmvar3):
#     da0 = ncarles_ds.data_vars[lesvar1]+ncarles_ds.data_vars[lesvar2]
#     da1 = xr.zeros_like(gotm_ds1.data_vars[gotmvar1].squeeze())
#     da2 = xr.zeros_like(gotm_ds2.data_vars[gotmvar1].squeeze())
#     da1.data = - gotm_ds1.data_vars[gotmvar1].squeeze().differentiate('z') * gotm_ds1.data_vars[gotmvar2].squeeze().interp(zi=gotm_ds1.z) + gotm_ds1.data_vars[gotmvar3].squeeze().interp(zi=gotm_ds1.z)
#     da2.data = - gotm_ds2.data_vars[gotmvar1].squeeze().differentiate('z') * gotm_ds2.data_vars[gotmvar2].squeeze().interp(zi=gotm_ds2.z) + gotm_ds2.data_vars[gotmvar3].squeeze().interp(zi=gotm_ds2.z)
#     for da in [da0, da1, da2]:
#         da.attrs['long_name'] = var
#         da.attrs['units'] = units
#     cmp_da(da0, da1, da2, var, units, levels, cmap)

In [ ]:
# T
levels = np.linspace(11.25, 11.85, 61)
plot_mean_fields('T', '$^\circ$C', levels, 'viridis', 'txym', 'temp', -273.15)

In [ ]:
# u
levels = np.linspace(-5, 5, 51)
plot_mean_fields('$\overline{u}/u_*$', 'unitless', levels, 'RdBu_r', 'uxym', 'u', scale=1/ustar)

In [ ]:
# v
levels = np.linspace(-10, 10, 51)
plot_mean_fields('$\overline{v}/u_*$', 'unitless', levels, 'RdBu_r', 'vxym', 'v', scale=1/ustar)

In [ ]:
levels = np.linspace(-12, 12, 61)
plot_turbulent_flux('wT', '$^\circ$C m/s', levels, 'RdBu_r', 'wtle', 'wtsb', 'temp', 'nuh', 'gamh', scale=1/Q0)

In [ ]:
levels = np.linspace(-1.2, 1.2, 41)
plot_turbulent_flux('$\overline{w^\prime u^\prime}/u_*^2$', 'unitless', levels, 'RdBu_r', 'uwle', 'uwsb', 'u', 'num', 'gamu', 1/ustar**2)

In [ ]:
levels = np.linspace(-1.2, 1.2, 41)
plot_turbulent_flux('$\overline{w^\prime v^\prime}/u_*^2$', 'unitless', levels, 'RdBu_r', 'vwle', 'vwsb', 'v', 'num', 'gamv', 1/ustar**2)